In [ ]:
%%capture
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 0
%config SqlMagic.displaycon = False

In [ ]:
import datetime

start = datetime.datetime.now() - datetime.timedelta(days=365)
end = datetime.datetime.now()
filterable_start = start.strftime('%Y-%m-%d')

In [ ]:
%%sql repeat_incident_locations <<

SELECT
    r.location_city_town as city,
    r.location_state as state,
    UPPER(r.location_name) as name,
    COUNT(*) count
FROM cts_forms_report r
WHERE location_name notnull
AND location_name != 'Other'
AND location_city_town notnull
AND location_state notnull
AND create_date >= '{{ start }}'
GROUP BY name, city, state
HAVING COUNT(*) > 2
ORDER BY count desc

In [ ]:
def make_data_div(city, state, location_name, count):
    location_citystate_dropdown.append(
        f'<option value="{city}-{state}">{city}, {state}</option>'
    )
    name = location_name.capitalize()
    return f"""<tr class="city-wrapper {city}-{state}">
                <td><span tabindex="0">{name}</span></td>
                <td><span tabindex="0">{city}, {state}</span></td>
                <td><a class="display-block" href="/form/view/?location_name={location_name}&location_city_town={city}&location_state={state}&create_date_start={filterable_start}&grouping=default&assigned_section=ADM&assigned_section=APP&assigned_section=CRM&assigned_section=DRS&assigned_section=ELS&assigned_section=EOS&assigned_section=FCS&assigned_section=HCE&assigned_section=IER&assigned_section=POL&assigned_section=SPL&assigned_section=VOT">
                    View {count} reports
                </a></td>
            </tr>
            """

In [1]:
from IPython.display import display, HTML
import numpy as np

readable_start = start.strftime("%B %d, %Y")
readable_end = end.strftime("%B %d, %Y")
incident_locations = repeat_incident_locations.DataFrame()
cities = incident_locations['city'].values
states = incident_locations['state'].values
location_names = incident_locations['name'].values
counts = incident_locations['count'].values
records = zip(cities, states, location_names, counts)
location_citystate_dropdown = []
location_name_divs = [
  make_data_div(city, state, location_name, count)
  for city, state, location_name, count in records
]

display(HTML(f"""
    <div class="crt-portal-card light maxh-mobile-lg overflow-y-scroll">
    <h3>Repeated incident location names by city</h3>
    <select name="incident_location_city" class="usa-select incident-location-city" aria-label="Incident location city">
        <option value="">Select a city</option>
        {''.join(sorted(np.unique(location_citystate_dropdown)))}
    </select>
    <p style="font-size: 1rem">
      <em>Repeated report locations submitted between
        <strong>{readable_start}</strong> and
        <strong>{readable_end}</strong>
      </em>
    </p>
    <p style="font-size: 1rem">
        <em>These results do not account for user typos, misspellings or abbreviations for location names.</em>
    </p>
    <br />
    <div class="data-table">
    <table class="usa-table crt-table">
     <thead class="districts">
        <tr>
        <th>
        <strong tabindex="0">Location Name</strong>
        </th>
        <th>
        <strong tabindex="0">City, State</strong>
        </th>
        <th>
        <strong tabindex="0"># of Reports</strong>
        </th>
        </tr>
    </thead>
    <tbody>
    {''.join(location_name_divs)}
    </tbody>
    </table>
    </div>
    </div>
"""))

ModuleNotFoundError: No module named 'numpy'